**Outline:**
- Introduce missing values
- Replace first TimeInterval with median value
- Split data into train/val/test

In [1]:
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle

In [2]:
from dateutil.parser import parse
from datetime import datetime
import time
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

pd.options.mode.chained_assignment = None #to run loop quicker without warnings

In [3]:
#name = 'bpi_2012'
name = 'bpi_2013'
#name = 'helpdesk'
args = {
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'input_dir': '../input/{}/'.format(name),  
    'nan_pct': 0.5,
    'train_pct': 0.7,
    'val_pct': 0.1,
}

args = argparse.Namespace(**args)

In [4]:
if not os.path.isdir('../input/'):
    os.makedirs('../input/')
    
if not os.path.isdir(args.input_dir):
    os.makedirs(args.input_dir)

In [5]:
sys.path.insert(0, './../utils/')
from utils import *

# Load data

In [6]:
# Only consider Case, Activity, Timestamp
cols = ['CaseID', 'Activity', 'CompleteTimestamp']

# For Timestamp: Convert to time
if name == 'helpdesk':
    data = pd.read_csv(args.data_dir + args.data_file)
else:
    data = pd.read_csv(args.data_dir + args.data_file, usecols=['Case ID', 'Activity', 'Complete Timestamp'])
    data['Case ID'] = data['Case ID'].apply(lambda x: x.split(' ')[1])
    

# Format for each column     
data.columns = cols
data['CompleteTimestamp'] = pd.to_datetime(data['CompleteTimestamp'], errors='coerce')
data['CaseID'] = data['CaseID'].apply(pd.to_numeric)

In [7]:
data.head()

,CaseID,Activity,CompleteTimestamp
0,1,Queued-Awaiting Assignment,2006-01-11 23:49:42
1,1,Accepted-In Progress,2012-03-15 19:53:52
2,1,Accepted-Assigned,2012-03-15 19:56:17
3,1,Accepted-In Progress,2012-03-15 20:09:05
4,1,Completed-Closed,2012-03-15 20:11:33


# Explore data

In [8]:
print('There are: {} cases'.format(len(data['CaseID'].unique())))
print('There are: {} activities'.format(len(data['Activity'].unique())))

There are: 1487 cases
There are: 7 activities


In [9]:
print('-----Frequency of different activities-----')
print(data['Activity'].value_counts())

-----Frequency of different activities-----
Accepted-In Progress          3066
Completed-Closed              1565
Queued-Awaiting Assignment     875
Accepted-Assigned              614
Accepted-Wait                  527
Unmatched-Unmatched             10
Completed-Cancelled              3
Name: Activity, dtype: int64


# Induce missing data

**To do:**
- nan_pct: percentage of nan values
- Induce missingness: 30% data

In [10]:
data.shape

(6660, 3)

In [11]:
total_NA = int(data.shape[0]*(data.shape[1]-1)*args.nan_pct)
print('Number of missing values: {}'.format(total_NA))

Number of missing values: 6660


In [12]:
# introduce missing Activities and Timestamps
missing_data = data.copy()
i = 0
while i < total_NA:
    row = np.random.randint(1, data.shape[0]) #exclude first row
    col = np.random.randint(1, data.shape[1]) #exclude CaseID
    if not pd.isnull(missing_data.iloc[row, col]): 
        missing_data.iloc[row, col] = np.nan
        i+=1

In [13]:
print('-----Frequency of different activities-----')
print(missing_data['Activity'].value_counts())

-----Frequency of different activities-----
Accepted-In Progress          1492
Completed-Closed               788
Queued-Awaiting Assignment     455
Accepted-Assigned              306
Accepted-Wait                  254
Unmatched-Unmatched              8
Completed-Cancelled              2
Name: Activity, dtype: int64


In [14]:
most_frequent_activity = missing_data['Activity'].value_counts().index[0]
print('Most frequent activity is: {}'.format(most_frequent_activity))

Most frequent activity is: Accepted-In Progress


In [15]:
first_timestamp = missing_data['CompleteTimestamp'][0]

# Impute the first TimeInverval

In [16]:
#missing_df = calculateTimeInterval(missing_data)
missing_df = calculateCumTimeInterval(missing_data)

#missing_df['TimeInterval'] = missing_df['TimeInterval'].apply(convert2seconds)
missing_df['CumTimeInterval'] = missing_df['CumTimeInterval'].apply(convert2seconds)

#missing_df['TimeInterval'] = missing_df['TimeInterval'].clip(lower=0)

In [17]:
#missing_median_duration_activity = {}
#missing_mean_duration_activity = {}

#for activity, group in missing_df.groupby(['Activity']):
#    missing_median_duration_activity[activity] = group['TimeInterval'].median()
#    missing_mean_duration_activity[activity] = group['TimeInterval'].mean()

In [18]:
#missing_median_duration_activity

In [19]:
# Replace first point with median
#missing_df['CumTimeInterval'].iloc[0] = missing_median_duration_activity[missing_df['Activity'].iloc[0]]

In [20]:
#missing_df.drop(['TimeInterval'], axis=1, inplace=True)

In [21]:
missing_df.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval
0,1,Queued-Awaiting Assignment,2006-01-11 23:49:42,0.0
1,1,NaN,2012-03-15 19:53:52,194817850.0
2,1,Accepted-Assigned,2012-03-15 19:56:17,194817995.0
3,1,Accepted-In Progress,NaT,NaN
4,1,Completed-Closed,NaT,NaN


# Split df to train/val/test

In [22]:
df = calculateCumTimeInterval(data)
df['CumTimeInterval'] = df['CumTimeInterval'].apply(convert2seconds)

In [23]:
df.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval
0,1,Queued-Awaiting Assignment,2006-01-11 23:49:42,0.0
1,1,Accepted-In Progress,2012-03-15 19:53:52,194817850.0
2,1,Accepted-Assigned,2012-03-15 19:56:17,194817995.0
3,1,Accepted-In Progress,2012-03-15 20:09:05,194818763.0
4,1,Completed-Closed,2012-03-15 20:11:33,194818911.0


In [24]:
groupByCase = df.groupby(['CaseID'])
missing_groupByCase = missing_df.groupby(['CaseID'])

# Split: 70% train, 10% validate, 20% test
train_size = int(len(groupByCase)*args.train_pct)
val_size = int(len(groupByCase)*args.val_pct)
test_size = len(groupByCase) - train_size - val_size

In [25]:
df.shape

(6660, 4)

In [26]:
df_train = pd.DataFrame(columns=list(df))
df_val = pd.DataFrame(columns=list(df))
df_test = pd.DataFrame(columns=list(df))

for caseid, data_case in groupByCase:
    if caseid <= train_size:
        df_train = df_train.append(data_case)
    elif train_size < caseid <= (train_size+val_size):
        df_val = df_val.append(data_case)
    else:
        df_test = df_test.append(data_case)

In [27]:
df.shape[0] == df_train.shape[0] + df_val.shape[0] + df_test.shape[0]

True

In [28]:
missing_df_train = pd.DataFrame(columns=list(missing_df))
missing_df_val = pd.DataFrame(columns=list(missing_df))
missing_df_test = pd.DataFrame(columns=list(missing_df))

#Note: case start from 1 not 0
for caseid, data_case in missing_groupByCase:
    if caseid <= train_size:
        missing_df_train = missing_df_train.append(data_case)
    elif train_size < caseid <= train_size+val_size:
        missing_df_val = missing_df_val.append(data_case)
    else:
        missing_df_test = missing_df_test.append(data_case)

In [29]:
missing_df.shape[0] == missing_df_train.shape[0] + missing_df_val.shape[0] + missing_df_test.shape[0]

True

In [30]:
len(df_train.groupby(['CaseID'])), len(df_val.groupby(['CaseID'])), len(df_test.groupby(['CaseID']))

(1040, 148, 299)

In [31]:
train_size, val_size, test_size

(1040, 148, 299)

In [32]:
len(missing_df_train.groupby(['CaseID'])), len(missing_df_val.groupby(['CaseID'])), len(missing_df_test.groupby(['CaseID']))

(1040, 148, 299)

In [33]:
#get number of rows
print(df_train.shape, df_val.shape, df_test.shape)
train_row_num = df_train.shape[0]
val_row_num = df_val.shape[0]
test_row_num = df_test.shape[0]

(4979, 4) (601, 4) (1080, 4)


In [34]:
missing_df_test.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval
5580,1189,NaN,2012-03-07 22:15:19,194135137.0
5581,1189,Completed-Closed,2012-03-09 16:57:30,194288868.0
5582,1189,NaN,2012-03-26 20:48:52,195771550.0
5583,1189,NaN,2012-03-26 20:49:58,195771616.0
5584,1189,Accepted-In Progress,NaT,NaN


In [35]:
avai_instance = 0
for row in range(len(missing_df_test)):
    if not pd.isnull(missing_df_test['CumTimeInterval'].iloc[row]) and not pd.isnull(missing_df_test['Activity'].iloc[row]):
        avai_instance+=1
        
print('Number of available row: {}'.format(avai_instance))

Number of available row: 251


In [36]:
nan_instance = 0
for row in range(len(missing_df_test)):
    if pd.isnull(missing_df_test['CumTimeInterval'].iloc[row]) or pd.isnull(missing_df_test['Activity'].iloc[row]):
        nan_instance+=1
        
print('Number of nan row: {}'.format(nan_instance))

Number of nan row: 829


In [37]:
missing_df_test.shape[0] == avai_instance + nan_instance

True

# Save df

In [38]:
df.to_csv(args.input_dir+'complete_df_full.csv', index=False)
missing_df.to_csv(args.input_dir+'missing_df_full.csv', index=False)

In [39]:
df_train.to_csv(args.input_dir+'complete_df_train.csv', index=False)
df_val.to_csv(args.input_dir+'complete_df_val.csv', index=False)
df_test.to_csv(args.input_dir+'complete_df_test.csv', index=False)

In [40]:
missing_df_train.to_csv(args.input_dir+'missing_df_train.csv', index=False)
missing_df_val.to_csv(args.input_dir+'missing_df_val.csv', index=False)
missing_df_test.to_csv(args.input_dir+'missing_df_test.csv', index=False)

In [41]:
pd.isnull(missing_df_test).sum()

CaseID                 0
Activity             558
CompleteTimestamp    552
CumTimeInterval      552
dtype: int64

# Save parameters

In [43]:
with open(args.input_dir + 'parameters.pkl', 'wb') as f: 
    pickle.dump(most_frequent_activity, f, protocol=2)
    pickle.dump(first_timestamp, f, protocol=2)
    pickle.dump(avai_instance, f, protocol=2)
    pickle.dump(nan_instance, f, protocol=2)
    pickle.dump(train_size, f, protocol=2)
    pickle.dump(val_size, f, protocol=2)
    pickle.dump(test_size, f, protocol=2)
    pickle.dump(train_row_num, f, protocol=2)
    pickle.dump(val_row_num, f, protocol=2)
    pickle.dump(test_row_num, f, protocol=2)